# **-->Mental Health Chatbot<--**


### 1. Requirments

In [2]:
!pip install streamlit textblob pandas huggingface pillow transformers deep_translator wget peft bitsandbytes asyncio

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 935.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.8 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=49ffc111a30a17120d43c56e64498b1b0951549c04cae581ade915eb063e9ab8
  Stored in directory: /

### 2. Model Download

In [3]:
! git clone https://huggingface.co/unsloth/Llama-3.2-1B-Instruct

Cloning into 'Llama-3.2-1B-Instruct'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 31 (delta 9), reused 0 (delta 0), pack-reused 3 (from 1)
Unpacking objects: 100% (31/31), 2.24 MiB | 2.44 MiB/s, done.


### **download Dataset**

In [4]:
! wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1qf0zh7YwXVSCNBxnRGmUD7GMMuEipiJ5' -O mentalhealth-1.json

--2024-10-15 17:50:31--  https://docs.google.com/uc?export=download&id=1qf0zh7YwXVSCNBxnRGmUD7GMMuEipiJ5
Resolving docs.google.com (docs.google.com)... 108.177.119.139, 108.177.119.138, 108.177.119.113, ...
Connecting to docs.google.com (docs.google.com)|108.177.119.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1qf0zh7YwXVSCNBxnRGmUD7GMMuEipiJ5&export=download [following]
--2024-10-15 17:50:31--  https://drive.usercontent.google.com/download?id=1qf0zh7YwXVSCNBxnRGmUD7GMMuEipiJ5&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.250.153.132, 2a00:1450:4013:c16::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.250.153.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4790520 (4.6M) [application/octet-stream]
Saving to: ‘mentalhealth-1.json’

mentalhealth-1.json 100%[===================>]   4.57M  --

### 3. Code of main APP

In [5]:
%%writefile app.py
import streamlit as st
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, PeftModel
import torch
from torch.utils.data import Dataset
from textblob import TextBlob
from deep_translator import GoogleTranslator
import asyncio
import pandas as pd
import plotly.express as px
from datetime import datetime
from typing import Dict, List, Any
import os

# Define the TRANSLATIONS dictionary (unchanged)
TRANSLATIONS = {
    "English": {
        "main_title": "Mental Health Chatbot",
        "initial_question": "Hello! How are you feeling today?",
        "input_placeholder": "Type your message here...",
        "thinking": "Thinking...",
        "how_feeling": "How are you feeling?",
        "log_mood": "Log Mood",
        "mood_logged": "Mood logged successfully!",
        "mood_over_time": "Mood Over Time",
        "your_name": "Your Name",
        "your_age": "Your Age",
        "your_interests": "Your Interests",
        "preferred_topics": "Preferred Topics",
        "about_chatbot": "About This Chatbot",
        "chatbot_description": "This chatbot is designed to provide mental health support...",
        "disclaimer": "Disclaimer: This chatbot is not a substitute for professional medical advice...",
        "crisis_response": "I'm concerned about what you've shared. Please reach out to a mental health professional or crisis helpline immediately.",
        "recommended_resources": "Recommended Resources",
        "footer": "© 2024 Mental Health Chatbot. All rights reserved.",
        "personalization": "Personalization",
        "mood_tracking": "Mood Tracking",
        "settings": "Settings",
        "select_language": "Select Response Language",
        "fine_tuning": "Fine-tuning",
        "fine_tune_button": "Fine-tune Model",
        "fine_tune_success": "Model fine-tuned successfully! Restart the app to use the fine-tuned model."
    }
    # Add translations for other languages here
}


@st.cache_resource
def load_model(model_name="Llama-3.2-1B-Instruct"):
    try:
        with st.spinner("Loading model. This may take a moment..."):
            tokenizer = AutoTokenizer.from_pretrained(model_name)

            if torch.cuda.is_available():
                st.info("CUDA is available. Loading model on GPU.")
                model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")
            else:
                st.warning("CUDA is not available. Loading model on CPU. This may be slower.")
                model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

        st.success("Model loaded successfully!")
        return tokenizer, model
    except Exception as e:
        st.error(f"Error loading model: {str(e)}")
        return None, None
class MentalHealthStructuredDataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        row = self.data.iloc[index]

        # Create input and target text using the conversational context and response
        input_text = f"Context: {row['Context']}"
        target_text = f"Response: {row['Response']}"

        # Tokenize the input and response texts
        inputs = self.tokenizer(
            input_text,
            truncation=True,
            max_length=self.max_length,
            padding="max_length",
            return_tensors="pt"
        )

        targets = self.tokenizer(
            target_text,
            truncation=True,
            max_length=self.max_length,
            padding="max_length",
            return_tensors="pt"
        )

        labels = targets["input_ids"].clone()
        labels[labels == self.tokenizer.pad_token_id] = -100  # Ignore padding tokens in loss calculation

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'labels': labels.flatten()
        }



def fine_tune_model(model, tokenizer, train_data, output_dir="./fine_tuned_model"):
    print(train_data.head())  # Debug print
    print(train_data.columns)  # Debug print

    # Set pad_token to eos_token if not already defined
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    # Manually specify target modules for Llama model
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"]

    lora_config = LoraConfig(
        r=16,
        lora_alpha=32,
        target_modules=target_modules,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM"
    )

    model = get_peft_model(model, lora_config)

    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=3,
        per_device_train_batch_size=4,
        warmup_steps=500,
        learning_rate=3e-4,
        fp16=True,
        logging_steps=10,
        save_strategy="epoch",
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=MentalHealthStructuredDataset(train_data, tokenizer, max_length=512),
    )

    trainer.train()
    model.save_pretrained(output_dir)
@st.cache_data
def generate_text(_tokenizer, _model, prompt: str, max_new_tokens: int = 150) -> str:
    try:
        inputs = _tokenizer(prompt, return_tensors="pt").to(_model.device)
        with torch.no_grad():
            outputs = _model.generate(**inputs, max_new_tokens=max_new_tokens, temperature=0.7, top_p=0.9)
        response = _tokenizer.decode(outputs[0], skip_special_tokens=True)
        return response[len(prompt):].strip()
    except Exception as e:
        st.error(f"Error generating text: {str(e)}")
        return "I'm sorry, I encountered an error while processing your request."

@st.cache_data
def analyze_sentiment(text: str) -> float:
    blob = TextBlob(text)
    return blob.sentiment.polarity

@st.cache_data
def translate_text(text: str, target_language: str) -> str:
    try:
        translator = GoogleTranslator(source='auto', target=target_language)
        return translator.translate(text)
    except Exception as e:
        st.error(f"Translation error: {str(e)}")
        return text

def translate_content(content: Dict[str, str], target_language: str) -> Dict[str, str]:
    if target_language == "English":
        return content
    translator = GoogleTranslator(source='en', target=target_language)
    return {k: translator.translate(v) for k, v in content.items()}

def get_translated_content(language: str) -> Dict[str, str]:
    if language not in TRANSLATIONS:
        content = TRANSLATIONS["English"]
        return translate_content(content, language)
    return TRANSLATIONS[language]

async def run_parallel(*functions):
    return await asyncio.gather(*[asyncio.to_thread(func) for func in functions])

def track_mood(t: Dict[str, str]):
    if 'mood_history' not in st.session_state:
        st.session_state.mood_history = []

    mood = st.multiselect(t["how_feeling"], ["Sad", "Happy", "Angry", "Anxious", "Feared"])
    if st.button(t["log_mood"]):
        st.session_state.mood_history.append({
            'date': datetime.now().strftime("%Y-%m-%d"),
            'mood': mood
        })
        st.success(t["mood_logged"])

    if st.session_state.mood_history:
        df = pd.DataFrame(st.session_state.mood_history)
        fig = px.line(df, x='date', y='mood', title=t["mood_over_time"])
        st.plotly_chart(fig)

def personalize_chatbot(t: Dict[str, str]) -> Dict[str, Any]:
    if 'user_preferences' not in st.session_state:
        st.session_state.user_preferences = {
            'name': '',
            'age': '',
            'interests': [],
            'preferred_topics': []
        }

    st.session_state.user_preferences['name'] = st.text_input(t["your_name"], st.session_state.user_preferences['name'])
    st.session_state.user_preferences['age'] = st.number_input(t["your_age"], min_value=0, max_value=120, value=st.session_state.user_preferences['age'] if st.session_state.user_preferences['age'] else 0)
    interests = st.multiselect(t["your_interests"], ["Reading", "Music", "Sports", "Art", "Travel", "Technology"], default=st.session_state.user_preferences['interests'])
    st.session_state.user_preferences['interests'] = interests
    topics = st.multiselect(t["preferred_topics"], ["Stress Management", "Positive Thinking", "Mindfulness", "Relationship Advice", "Career Guidance"], default=st.session_state.user_preferences['preferred_topics'])
    st.session_state.user_preferences['preferred_topics'] = topics

    return st.session_state.user_preferences

def recommend_resources(sentiment: float) -> List[str]:
    resources = {
        "positive": [
            "Mindfulness meditation app",
            "Gratitude journaling guide",
            "Positive affirmations list"
        ],
        "neutral": [
            "Self-care checklist",
            "Stress management techniques",
            "Healthy habit tracker"
        ],
        "negative": [
            "Crisis helpline numbers",
            "Therapy finder tool",
            "Coping strategies for difficult emotions"
        ]
    }

    if sentiment > 0.2:
        category = "positive"
    elif sentiment < -0.2:
        category = "negative"
    else:
        category = "neutral"

    return resources[category]

def detect_crisis(text: str) -> bool:
    crisis_keywords = ["suicide", "kill myself", "want to die", "end it all"]
    return any(keyword in text.lower() for keyword in crisis_keywords)

def run_app():
    st.set_page_config(page_title="Mental Health Chatbot", page_icon="🤖", layout="wide")

    # Initialize translations in session state
    if 'translations' not in st.session_state:
        st.session_state.translations = TRANSLATIONS["English"]

    tokenizer, base_model = load_model()
    if tokenizer is None or base_model is None:
        st.error("Failed to load the model. Please check the error message above and try again.")
        st.stop()

    # Check if a fine-tuned model exists
    fine_tuned_model_path = "./fine_tuned_model"
    if os.path.exists(fine_tuned_model_path):
        st.info("Loading fine-tuned model...")
        model = PeftModel.from_pretrained(base_model, fine_tuned_model_path)
        st.success("Fine-tuned model loaded successfully!")
    else:
        model = base_model
        st.warning("No fine-tuned model found. Using base model.")

    with st.sidebar:
        st.title(st.session_state.translations["settings"])
        language = st.selectbox(st.session_state.translations["select_language"], ["English", "Hindi", "Gujarati", "Marathi"])
        language_code = {"English": "en", "Hindi": "hi", "Gujarati": "gu", "Marathi": "mr"}[language]

        if 'language' not in st.session_state or st.session_state.language != language:
            st.session_state.language = language
            st.session_state.translations = get_translated_content(language_code)
            st.rerun()

        t = st.session_state.translations

        st.divider()
        st.subheader(t["personalization"])
        user_preferences = personalize_chatbot(t)
        st.divider()
        st.subheader(t["mood_tracking"])
        track_mood(t)
        st.divider()
        st.write(t["about_chatbot"])
        st.write(t["chatbot_description"])
        st.write(t["disclaimer"])

        # Add fine-tuning option
        st.divider()
        st.subheader(t["fine_tuning"])
        if st.button(t["fine_tune_button"]):
            train_data = pd.read_json('mentalhealth-1.json', lines=True)
            fine_tune_model(base_model, tokenizer, train_data)
            st.success(t["fine_tune_success"])

    st.title(t["main_title"])

    if 'messages' not in st.session_state:
        st.session_state.messages = []
    if 'conversation_started' not in st.session_state:
        st.session_state.conversation_started = False

    if not st.session_state.conversation_started:
        initial_question = t["initial_question"]
        st.session_state.messages.append({"role": "assistant", "content": initial_question})
        st.session_state.conversation_started = True

    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

    user_input = st.chat_input(t["input_placeholder"])

    if user_input:
        st.session_state.messages.append({"role": "user", "content": user_input})
        with st.chat_message("user"):
            st.markdown(user_input)

        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        user_input_en, sentiment_score = loop.run_until_complete(run_parallel(
            lambda: translate_text(user_input, 'en') if language != "English" else user_input,
            lambda: analyze_sentiment(user_input)
        ))

        sentiment = "positive" if sentiment_score > 0 else "neutral" if sentiment_score == 0 else "negative"

        if detect_crisis(user_input_en):
            crisis_response = t["crisis_response"]
            st.session_state.messages.append({"role": "assistant", "content": crisis_response})
            with st.chat_message("assistant"):
                st.markdown(crisis_response)
            st.stop()

        formatted_history = "\n".join(
            f"{'Assistant' if msg['role'] == 'assistant' else 'Human'}: {msg['content']}"
            for msg in st.session_state.messages[-5:]
        )

        response_prompt = f"""You are a compassionate AI assistant dedicated to supporting mental well-being. Based on the conversation history and the user's current sentiment, craft a thoughtful response that addresses the user's input and provides support. Then, ask a follow-up question to encourage further discussion.

User Information:
Name: {user_preferences['name']}
Age: {user_preferences['age']}
Interests: {', '.join(user_preferences['interests'])}
Preferred Topics: {', '.join(user_preferences['preferred_topics'])}

Conversation History:
{formatted_history}

User's Input:
Human: {user_input_en}

Detected Sentiment: {sentiment}

Your Response with emoji and Follow-up Question:
Assistant:"""
        with st.spinner(t["thinking"]):
            response_en = generate_text(tokenizer, model, response_prompt)
            response = translate_text(response_en, language_code) if language != "English" else response_en
            st.session_state.messages.append({"role": "assistant", "content": response})

            with st.chat_message("assistant"):
                st.markdown(response)

            resources = recommend_resources(sentiment_score)
            st.subheader(t["recommended_resources"])
            for resource in resources:
                st.write(f"- {resource}")

    st.markdown('</div>', unsafe_allow_html=True)
    st.markdown("---")
    st.markdown(t["footer"])

if __name__ == "__main__":
    run_app()



Writing app.py


### 4. To run App on Google Colab

In [6]:
!wget -q -O - ipv4.icanhazip.com

34.32.171.79


In [ ]:
# prompt: start streamlit app

! streamlit run /content/app.py & npx localtunnel --port 8501



Need to install the following packages:
  localtunnel@2.0.2
Ok to proceed? (y) 
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.32.171.79:8501

y
your url is: https://quiet-parrots-hang.loca.lt
2024-10-15 17:51:33.455435: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-15 17:51:33.801985: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-15 17:51:33.896765: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-15 17:51:34.483035: I tensorflow/core/platform/cpu_fe

### 4. To run App on Device

In [1]:
! streamlit run app.py

/bin/bash: line 1: streamlit: command not found
